In [1]:
#All the necessary imports
import numpy as np
import nltk
import pandas as pd
import os
import re
import csv
import gzip

### This is an example of a movie review.

In [3]:
f = open('./aclImdb/train/pos/10327_7.txt', encoding="utf8")
print(f.read())

Given the title, this first follow-up to QUARTET (1948) obviously reduces the number of W. Somerset Maugham stories which comprise the film. The author still turns up to introduce the episodes, but there’s no epilogue this time around; by the way, while the script of the original compendium gave sole credit to R.C. Sheriff, here Maugham himself also lent a hand in the adaptation, as well as Noel Langley (though it’s unclear whether they contributed one segment each or else worked in unison). As can be expected, much of the crew of QUARTET has been retained for the second installment – though this also extends to at least three cast members, namely Naunton Wayne, Wilfrid Hyde-White and Felix Aylmer (the last two had bit parts in the episode from QUARTET entitled “The Colonel’s Lady”). While TRIO ultimately emerges to be a lesser achievement than its predecessor (slightly unbalanced by the third story which takes up more than half the running-time), it’s still done with the utmost care, 

#### Putting the train and test data into a pandas dataframe. It shuffles the data as well.

In [2]:
directory = "./aclImdb" #Make sure you put the data folder in the same directory as this jupyter notebook file
labeledData = {}
for i in ["train", "test"]:
    labeledData[i] = []
    for sentiment in ["pos", "neg"]:
        score = 1 if sentiment == "pos" else 0
        path = os.path.join(directory, i, sentiment)
        for filename in os.listdir(path):
            with open(os.path.join(path, filename), encoding="utf8") as f:
                labeledData[i].append([f.read(), score])  #Initially adds them to separate lists

np.random.shuffle(labeledData["train"]) #Shuffling
labeledData["train"] = pd.DataFrame(labeledData["train"], columns = ['text', 'sentiment']) #Putting them in a dataframe
np.random.shuffle(labeledData["test"])
labeledData["test"] = pd.DataFrame(labeledData["test"], columns = ['text', 'sentiment'])
labeledData["train"], labeledData["test"] #Prints out both pandas dataframes


(                                                    text  sentiment
 0      Absolutely enjoyable singing and dancing movie...          1
 1      After all these years I still consider this se...          1
 2      A hilarious and insightful perspective of the ...          1
 3      After reading all of the rave reviews about th...          0
 4      "Tintin and I" first of all struck me as a mas...          1
 5      The latest film by the Spanish director Agusti...          1
 6      Okay, I absolutely LOVE Ben Stiller, although ...          0
 7      Okay, like many other such films, spawned out ...          1
 8      If the very thought of Arthur Askey twists you...          1
 9      Jean-Claude Van Damme plays twin brothers Alex...          0
 10     "The Invisible Mouse" is a delightful and diff...          1
 11     Greatly enjoyed this 1945 mystery thriller fil...          1
 12     I must admit I am a big fan of South Park and ...          1
 13     I was honestly surprised b

In [4]:
train = labeledData["train"].to_csv(r'C:\Users\mdodda3-gtri\Untitled Folder\train.csv.gz', index = None, header=False, compression='gzip')
test = labeledData["test"].to_csv(r'C:\Users\mdodda3-gtri\Untitled Folder\test.csv.gz', index = None, header=False, compression='gzip')


In [5]:

# train1 = labeledData["train"].iloc[:6250].to_csv (r'C:\Users\mdodda3-gtri\Untitled Folder\train1.csv.gz', index = None, header=False, compression='gzip')
# train2 = labeledData["train"].iloc[6250:12500].to_csv (r'C:\Users\mdodda3-gtri\Untitled Folder\train2.csv.gz', index = None, header=False, compression='gzip')
# train3 = labeledData["train"].iloc[12500:18750].to_csv (r'C:\Users\mdodda3-gtri\Untitled Folder\train3.csv.gz', index = None, header=False, compression='gzip')
# train4 = labeledData["train"].iloc[18750:].to_csv (r'C:\Users\mdodda3-gtri\Untitled Folder\train4.csv.gz', index = None, header=False, compression='gzip')

# test1 = labeledData["test"].iloc[:6250].to_csv (r'C:\Users\mdodda3-gtri\Untitled Folder\test1.csv.gz', index = None, header=False, compression='gzip')
# test2 = labeledData["test"].iloc[6250:12500].to_csv (r'C:\Users\mdodda3-gtri\Untitled Folder\test2.csv.gz', index = None, header=False, compression='gzip')
# test3 = labeledData["test"].iloc[12500:18750].to_csv (r'C:\Users\mdodda3-gtri\Untitled Folder\test3.csv.gz', index = None, header=False, compression='gzip')
# test4 = labeledData["test"].iloc[18750:].to_csv (r'C:\Users\mdodda3-gtri\Untitled Folder\test4.csv.gz', index = None, header=False, compression='gzip')


The first column contains the movie reviews in separated rows.
The second column indicates whether the review is a positive or negative review. 
A positive reivew has 7-10 stars, A negative review has 1-4 stars. 5-6 stars are disregarded

0        Since the day I saw this film when it came out...
1        I read the book in 5th grade and now a few yea...
2        In France, it's considered polite from French ...
3        REALLY? REALLY???? I know if you make a politi...
4        If you're going to put on a play within the pr...
5        This movie is great! Brad Pitt will never be a...
6        With a minimal budget, a running time of eight...
7        Etienne Girardot is just a character actor--th...
8        I don't know whether to recommend this movie t...
9        I saw this when on The Wonderful World of Disn...
10       Wow. I have seen some really bad movies in my ...
11       What's not to like about this movie? Every yea...
12       The movie was actually a romantic drama based ...
13       This is a plain old spooky ghost story. I show...
14       This dumb comedy really does a good job of was...
15       This had a great cast with big-name stars like...
16       This is a must-see documentary movie for anyon.

In [72]:

total = np.vstack((labeledData["train"]["text"], labeledData["test"]["text"]))
#total = np.array([labeledData["test"]["text"]])
for col in total:
    print(col.shape)
total.reshape(1,total.shape[1] * total.shape[0]).shape

(25000,)
(25000,)


(1, 50000)

In [7]:
type(pd.DataFrame(labeledData["train"]["text"].values))

pandas.core.frame.DataFrame

In [28]:
import nltk
from nltk import  word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer, SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
stemmer =SnowballStemmer("english")
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]


## Converting the movie reviews into Specific Bag of Words Vectors

In [38]:
#sklearn imports

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC, NuSVC, SVC
from sklearn.linear_model import OrthogonalMatchingPursuit, LogisticRegression, SGDClassifier, PassiveAggressiveClassifier, Perceptron, RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, ComplementNB
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.feature_extraction import stop_words as stopwords



#Stop word list created to be used
stop_words = ['in', 'of', 'at', 'a', 'the']

#how each review will be vectorized
vectorizer = CountVectorizer(stop_words=stop_words, #These stop words are removed
                             binary=True, # if it contains in list it's 1, else it is 0
                             ngram_range=(1,2), #contains pairs of words as well
                            
                            )
newVect = CountVectorizer (tokenizer = LemmaTokenizer(),
                           stop_words=stop_words,
                           binary=True,
                           ngram_range=(1,2))



x_train = vectorizer.fit_transform(labeledData["train"]["text"]) #reviews from train dataframe are vectorized
x_test = vectorizer.transform(labeledData["test"]["text"])  #reviews from test dataframe are vectorized




(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)


(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)


(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)


(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)


(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)


(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)


array([['Since the day I saw this film when it came out in 1981, it has been one of my top 3 favorites. The blurb I wrote for Amazon is below, and I\'m just thrilled that it\'s finally coming out on DVD on 10/17/06 - the film\'s 25th anniversary.<br /><br />The last credit in this film explains its appeal - "Thank you to the people of Manhattan on whose island this was filmed." A charming and witty romantic comedy, it is a love story written to New Yorkers (Peter Bogdanovich is a native) who can identify every location (West 12th Street, the Ansonia, the old FAO Schwartz, the Plaza, the Roxy, Chez Brigitte, and City Limits which was a country & western club). One gets the impression that the entire ensemble cast clicked as well off-screen as they do on, and this intimacy is clearly communicated. I laughed, I cried, it was better than CATS. Not only an ode to Dorothy Stratten, it was also Audrey Hepburn\'s last feature appearance (she had a cameo subsequent to this film) and her inner b

In [11]:
from keras.preprocessing.text import Tokenizer, one_hot, text_to_word_sequence
import keras
import gensim
train_data = labeledData["train"]["text"]
test_data = labeledData["test"]["text"]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data)
#x_train = tokenizer.texts_to_sequences(train_data)
#x_test = tokenizer.texts_to_sequences(test_data)
type(x_train)


Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


scipy.sparse.csr.csr_matrix

In [12]:
#x_train
#x_train = np.array([np.array(xi) for xi in x_train])
#x_test = np.array([np.array(xi) for xi in x_test])
len(tokenizer.word_index)

88582

In [15]:
# Xencoded = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=200, padding='post')
# XencodedTest = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=200, padding = 'post')

# model = keras.models.Sequential()
# embedding = keras.layers.embeddings.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=300, input_length=200, trainable=True, mask_zero=True)
# model.add(embedding)
# model.add(keras.layers.LSTM(units=150, dropout=0.2, recurrent_dropout=0.2, return_sequences=False))
# model.add(keras.layers.Dense(2, activation='softmax'))
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# print(model.summary())



In [16]:

# from sklearn.model_selection import StratifiedShuffleSplit
# ss = StratifiedShuffleSplit(n_splits = 1, test_size= 0.2, random_state=1).split(Xencoded, labeledData[""])

In [90]:
# train_labels = keras.utils.to_categorical(labeledData["train"]["sentiment"], 2)
# test_labels = keras.utils.to_categorical(labeledData["test"]["sentiment"], 2)
# early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=2, mode='auto', restore_best_weights=False)
# history = model.fit(x=x_train, y=train_labels, epochs=50, batch_size=32, shuffle=True, validation_data = (XencodedTest, test_labels), verbose=2, callbacks=[early_stop])
# predicted = model.predict(test_x, verbose=2)
# predicted_labels = predicted.argmax(axis=1)

l = ["34", "34"]
l=["d"]
l = ''.join(l)

a = np.array([l], dtype = object)
b = np.array(["ldkjsf"], dtype = object)
np.vstack([a,b])

array([['d'],
       ['ldkjsf']], dtype=object)

## Creating a model, training, and applying the model

In [18]:
#Any linear model can be used
model = LinearSVC(C=1)  
#This is another model that can be used
model2 = LogisticRegression()   
model4 = SGDClassifier()
model5 = PassiveAggressiveClassifier()
model6 = NuSVC()
model6 = NearestCentroid()
#model trained associating review vectors to it sentiment scores 
model.fit(x_train, labeledData["train"]["sentiment"]) 
#applying model on test data creates what model thinks is sentiment scores associated with each movie review
y_pred = model.predict(x_test) 
#accuracy score created by comparing to actual sentiment score to model's predicted sentiment score
acc = accuracy_score(labeledData["test"]["sentiment"], y_pred)
print("Accuracy score of model: "+str(acc))

Accuracy score of model: 0.89516


In [19]:
from sklearn.metrics import confusion_matrix as cm
cm(labeledData["test"]["sentiment"], y_pred)

array([[11154,  1346],
       [ 1275, 11225]], dtype=int64)

#This is a the representation of the matrix of all the vectorized movie reviews. 
#The data is very sparse, so this data only shows the position on if a word is contained in a movie review
#The first number in the ordered pair represents the movie review. 
#The second number in the ordered pair represents a particular word.
#The row of 1 shows that it is the word is there in the movie review.
#Everthing else are 0s

In [ ]:
print(np.hstack((x_train[0], np.array(1))))

In [ ]:
for i in range(5):
    print(i)

In [ ]:
#This prints out the first 10 n-grams associated with the first review above and the correspoding indeces for each n-gram
count = 10;
for i in x_train[0].nonzero()[1]:
    print(str(i)+": " + vectorizer.get_feature_names()[i])
    count-=1
    if (count == 0):
        break

In [ ]:
from gensim.models import Word2Vec
#gzip.open("C:\Users\mdodda3-gtri\emade\datasets\movie_reviews\GoogleNews-vectors-negative300.bin.gz")
wv = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)
wv.init_sims(replace=True)


## Using pipeline to make it easier (puts, vectorizer and model in one line)
Using Tfidf by transforming original countvectorizer using Tfidf

In [165]:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(labeledData["train"]["text"])
# x_train = tokenizer.texts_to_sequences(labeledData["train"]["text"])
# x_test =  tokenizer.texts_to_sequences(labeledData["train"]["text"])

import time

start = time.time()




stop_words = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than']
stop_words = ['in', 'of', 'at', 'a', 'the']
def tokenize(val):
    ans = text_to_word_sequence(val)
    ans = [a for a in ans if not a in stop_words]
    return ans
reviews = labeledData["train"]["text"].append(labeledData["test"]["text"], ignore_index=True).values

words = [tokenize(val) for val in reviews.tolist()]

model = gensim.models.Word2Vec(sentences=words, size=100, window = 1000, workers = 4, min_count=1, sg = 0)

def method(list, wv):
    mean = []
    for word in list:
        if word in wv.vocab:
            mean.append(wv[word])
        else:
            a.append(word)
    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

In [166]:
#x_train = np.array([method(review, wv) for review in reviews.tolist()])
#x_train.shape
x_train = words[:25000]
x_train = np.array([method(review, model.wv) for review in x_train])
x_test = words[25000:]
x_test = np.array([method(review, model.wv) for review in x_test])

In [163]:
#.85428
#159.6575005054474 seconds
## CBOW, size changing
#10: .75536
#25: 
#50: .84576
#100:.8596 ~40
#500: .87052 ~76
#750: .87104
#1000: .87128 ~116.53

##CBOW window
#1 .833
#5 .8501
#10 .86 ~45
#25 .87 ~ 57
#50 same as 25
#100 .89 105
#200 .887 140
#500 .892
#1000 .894

##Skip-Gram
#25: .85
#50: .87464
#100: .88 ~ 154
#200: .89 ~ 190
#500:  .89 322   ~ (300)

In [164]:
model = LinearSVC()

model.fit(x_train, labeledData["train"]["sentiment"]) 
#applying model on test data creates what model thinks is sentiment scores associated with each movie review
y_pred = model.predict(x_test) 
#accuracy score created by comparing to actual sentiment score to model's predicted sentiment score
acc = accuracy_score(labeledData["test"]["sentiment"], y_pred)
print("Accuracy score of model: "+str(acc))
end = time.time()
print(end - start)

Accuracy score of model: 0.89452
224.66473388671875


In [ ]:
tokenizer.word_index
#wv.syn0norm
#wv.wv.most_similar("little")
# for word, i in tokenizer.items():
wv.vocab["hello"].count
wv["hello"]
tokenizer.word_index

def tokenize(val):
    ans = text_to_word_sequence(val)
    ans = [a for a in ans if not a in stop_words]
    return ans

x_train_list = [tokenize(val) for val in labeledData["train"]["text"].values.tolist()]
x_test_list = [tokenize(val) for val in labeledData["test"]["text"].values.tolist()]
a =[]
def method(list, wv):
    mean = []
    for word in list:
        if word in wv.vocab:
            mean.append(wv[word])
        else:
            a.append(word)
    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean
x_train = np.array([method(review, wv) for review in x_train_list])
x_test = np.array([method(review, wv) for review in x_test_list])


model = LinearSVC()

model.fit(x_train, labeledData["train"]["sentiment"]) 
#applying model on test data creates what model thinks is sentiment scores associated with each movie review
y_pred = model.predict(x_test) 
#accuracy score created by comparing to actual sentiment score to model's predicted sentiment score
acc = accuracy_score(labeledData["test"]["sentiment"], y_pred)
print("Accuracy score of model: "+str(acc))

In [ ]:
x_train_list[0]
print(a[89])

In [ ]:
# # nltk.download('punkt')
# # [nltk.word_tokenize(sentences) for sentences in train]
# # model = gensim.models.Word2Vec(sentences = )
# from keras.preprocessing.sequence import pad_sequences
# wv.wv.most_similar('horrible')



# tokenizer_obj = Tokenizer()
# tokenizer_obj.fit_on_texts(labeledData["train"]["text"])
# sequences = tokenizer_obj.texts_to_sequences(labeledData["train"]["text"])
# word_index = tokenizer_obj.word_index
# max_length = max([len(s.split()) for s in labeledData["train"]["text"] + labeledData["test"]["text"]])
# review_pad = pad_sequences(sequences, maxlen = max_length)

# num_words = len(word_index) + 1
# embedding_matrix = np.zeros((num_words, 100))

# for word, i in word_index.items():
#     if i > num_words:
#         continue
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector
# type(embedding_matrix)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier, AdaBoostClassifier,ExtraTreesClassifier


#pipe = Pipeline([('vect', CountVectorizer(binary=True, ngram_range=(1,2))),('clf',LogisticRegression(C=.05))])fg
stop_words = ['in', 'of', 'at', 'a', 'the']

##pipe = Pipeline([('vect', CountVectorizer(binary=True, ngram_range=(1,2), stop_words=stop_words)), ('clf',LinearSVC(C=.01))])##

#Making pipeline. has vectorizer, uses tfidf, and uses multinomialnb for the model
pipe1 = Pipeline([('vect', CountVectorizer(tokenizer = LemmaTokenizer(), binary=True, ngram_range=(1,2), stop_words=stop_words)), 
                 ('clf',SGDClassifier())])
pipe2 = Pipeline([('vect', CountVectorizer(stop_words=stop_words, binary=True, ngram_range=(1,2) )), ('clf', SGDClassifier())])

In [ ]:
#fits whole pipeline using the train text as x value and train sentiment as y value and makes model
for pipe in [pipe1, pipe2]:
    pipe.fit(labeledData["train"]["text"], labeledData["train"]["sentiment"]) 
    #model predicts sentiment scores using test dataframe
    y_pred = pipe.predict(labeledData["test"]["text"])
    #compares model sentiment scores to actual sentiment scores
    acc = accuracy_score(labeledData["test"]["sentiment"], y_pred)
    print("Accuracy score of the model: "+ str(acc))

In [ ]:
# from gensim.models import Word2Vec
# wv = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)
# wv.init_sims(replace=True)

In [159]:
n =np.array([1])

In [160]:
np.array([n])

array([[[1]]])